## Publish a Data Product
In this notebook we want to publish the payment delay prediction results to the data catalog of SAP Business Data Cloud. For that we need to build a custom data product by using the python library sap-bdc-connect-sdk. We will also utilize the Delta Share protocol, which allows us to share the data product without the need of copying the result table. The result table remains persisted in SAP Databricks, and will be remotely accessible from the data catalog.

The following steps need to be applied in this exercise:

1. Install SAP SDK
2. Create client
3. Create Delta Share
4. Add recipient to share
5. Create ORD object
6. Create CSN object
7. Publish data product

%md
> **Pre-requisite: SECRET SCOPE**
>
> This step has already been done by the administrator to facilitate the sharing process. However, if you need to do > it on you own you can follow the steps described here to create a secret scope:
>
> To create a secret scope you can either use the following URL https://<databricks-instance>#secrets/createScope. 
> Replace <databricks-instance> with the workspace URL of your Databricks deployment.
>
> Alternatively, you can run the following command in the terminal by clicking on the terminal icon on the lower  right corner: `databricks secrets create-scope sap-bdc-connect-sdk`. 
>
>The secret scope only has to be created once and can be made accessible to all workspace users by either toggling  `manage principal` to `all workspace users` or via the terminal using the following command `databricks secrets put-acl sap-bdc-connect-sdk users READ`. To check whether the assignment worked, you can then use the command `databricks secrets list-acls sap-bdc-connect-sdk`.
>
> A full explanation can be found here https://docs.databricks.com/aws/en/security/secrets/example-secret-workflow

### 1. Install and Load Packages
To be able to share the enhanced data products back to SAP Business Data Cloud, we need the `SAP SDK` (https://pypi.org/project/sap-bdc-connect-sdk/) 

In [0]:
%pip install sap-bdc-connect-sdk
%pip install --upgrade pydantic
%restart_python

In [0]:
pip show sap-bdc-connect-sdk

### 2. Create a Client
The `DatabricksClient` receives dbutils as a parameter, which is a Databricks utility that can be used inside the Databricks notebooks.

The `BdcConnectClient` receives the DatabricksClient as a parameter to get information from the Databricks environment (e.g. secrets, api_token, workspace_url_base)

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient
from bdc_connect_sdk.utils import csn_generator
import json

databricks_client = DatabricksClient(dbutils)
bdc_connect_client = BdcConnectClient(databricks_client)

### 3. Create Delta Share
A share is a mechanism for distributing and accessing data across different systems. Creating or updating a share involves including specific attributes, such as @openResourceDiscoveryV1, in the request body, aligning with the Open Resource Discovery protocol. This procedure ensures that the share is properly structured and described according to specified standards, facilitating effective data sharing and management. This is done in step 5.
# &#x270D;
Please replace the values `<CATALOG_NAME>` and `<SCHEMA_NAME>` with the specific values that match our use case and group. You can find the correct names by checking the **Unity Catalog** and look for the specific catalog and schema names: `uc_XXX`, `grpX`.

In [0]:
%sql
-- CREATE CATALOG IF NOT EXISTS <CATALOG_NAME>;
SET CATALOG <CATALOG_NAME>;
CREATE SCHEMA IF NOT EXISTS <SCHEMA_NAME>;
USE SCHEMA <SCHEMA_NAME>;

# &#x270D;
Now let's create the Delta Share and include the two results tables created in the schema into the Delta Share. 

Please replace variable `<SHARE_NAME>` with the value `grpx_share_delayed_payments`, and the variables `<DELAY_PREDICTION_SHAP_TABLE>` and `<DELAY_PREDiCTION_EXPLAIN_TABLE>` with the corresponding table names of the Unity Catalog. 

In [0]:
%sql
CREATE SHARE IF NOT EXISTS <SHARE_NAME>;
ALTER SHARE <SHARE_NAME> ADD TABLE <DELAY_PREDICTION_SHAP_TABLE> WITH HISTORY;
ALTER SHARE <SHARE_NAME> ADD TABLE <DELAY_PREDiCTION_EXPLAIN_TABLE> WITH HISTORY;

### 4. Add Recipient to Share
# &#x270D;
To be able to share the data product to BDC we need to set `sap-business-data-cloud` as the recipient for the delta share we've just created. Please replace the variable `<RECIPIENT_NAME>` with `sap-business-data-cloud`. 

In [0]:
%sql
GRANT SELECT ON SHARE <SHARE_NAME> TO RECIPIENT `<RECIPIENT_NAME>`;

Remark: Alternatively, this can be done by heading over to the **Unity Catalog** > **Delta Sharing** > **Shared by me** > *Select Share* > **Recipient** > **Add Recipients** > *Select sap-business-data-cloud*

### 5. Create ORD object 



Creating or updating a share involves including specific attributes, such as @openResourceDiscoveryV1, in the request body, aligning with the [Open Resource Discovery protocol](https://open-resource-discovery.github.io/specification/). This procedure ensures that the share is properly structured and described according to specified standards, facilitating effective data sharing and management. 
# &#x270D;
In the following code you'll need to set the parameters for the share:
- `<DATA_PRODUCT_NAME>` : 
    - e.g. "grpxx Delayed Payment Prediction and Explanation"
- `<SHORT_DESCRIPTION>` : 
    - e.g. "grpxx Data asset for payment predictions"
- `<LONG_DESCRIPTION>`: 
    - e.g. "This data product contains the data used for the delayed payment prediction model. Furthermore, SHAP values have been attached to store the impact/contribution of the features a payment delay prediction.Another table contains the explanation of the predictions in human natural language"

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "<RECIPIENT_NAME>"))

share_name = "<SHARE_NAME>"

open_resource_discovery_information = {
    "@openResourceDiscoveryV1": {
        "title": "<DATA_PRODUCT_NAME>",
        "shortDescription": "<SHORT_DESCRIPTION>",
        "description": "<LONG_DESCRIPTION>"
    }
}

response = bdc_connect_client.create_or_update_share(
    share_name,
    open_resource_discovery_information
)
print(f"[REQUEST] Create or update share request was executed and returned {response}")

### 6. Create CSN object

The CSN serves as a standardized format for configuring and describing shares within a network. To create or update the CSN for a share, it's advised to prepare the CSN content in a separate file and include this content in the request body. This approach ensures accuracy and compliance with the CSN interoperability specifications, facilitating consistent and effective share configuration across systems.
# &#x270D;
In the following code you'll need to set the parameters `<RECIPIENT_NAME>` and `<SHARE_NAME>` for the share.

> [IMPORTANT] removing column `sharp_array` from the table `<DELAY_PREDICTION_SHAP_TABLE>` before sharing, since as of today, SAP Datasphere can not handle the list data type

In [0]:
%sql
ALTER TABLE <DELAY_PREDICTION_SHAP_TABLE> DROP COLUMN shap_array;

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient
from bdc_connect_sdk.utils import csn_generator

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "<RECIPIENT_NAME>"))

share_name = "<SHARE_NAME>"

csn_schema = csn_generator.generate_csn_template(share_name)

response = bdc_connect_client.create_or_update_share_csn(
    share_name,
    csn_schema
)
print(f"[REQUEST] Create or update CSN request was executed and returned {response if response else 'OK'}")

### 7. Publish a Data Product
A Data Product is an abstraction that represents a type of data or data set within a system, facilitating easier management and sharing across different platforms. It bundles resources or API endpoints to enable efficient data access and utilization by integrated systems. Publishing a Data Product allows these systems to access and consume the data, ensuring seamless communication and resource sharing.
# &#x270D;
In the following code you'll need to set the parameters `<RECIPIENT_NAME>` and `<SHARE_NAME>` for the share.

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient

bdc_connect_client = BdcConnectClient(DatabricksClient(dbutils, "<RECIPIENT_NAME>"))

share_name = "<SHARE_NAME>"

response = bdc_connect_client.publish_data_product(
    share_name
)
print(f"[REQUEST] Publish Data Product request was executed and returned {response if response else 'OK'}")